In [1]:
pip install pytorch-pretrained-bert transformers

     |████████████████████████████████| 133kB 2.9MB/s 
     |████████████████████████████████| 573kB 8.0MB/s 
     |████████████████████████████████| 890kB 13.8MB/s 
     |████████████████████████████████| 3.7MB 20.6MB/s 
     |████████████████████████████████| 1.0MB 36.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=3782f17d17250b5f899495c6dcaefc446f61692790d45fbde70749a19d732a8d
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [2]:
import torch 

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2, output_attentions=False, output_hidden_states=False)

model.cuda()

optimizer = AdamW(model.parameters(), 
                lr=2e-5, # learning rate, default = 5e-5
                eps=1e-8 # adam_epsilon, default = 1e-8
                )

100%|██████████| 995526/995526 [00:00<00:00, 12283199.88B/s]


In [14]:
# Load the training sentences
print('Loading the training data...')

import pandas as pd

CSV_FILE_PATH = './drive/My Drive/Colab Notebooks/diploma_data/lgbt_homophobia_final.csv'

df = pd.read_csv(CSV_FILE_PATH, delimiter=',', header=None, names=['comment', 'label'])

print('Number of comments: {:,}\n'.format(df.shape[0]))
df.sample(5)

Loading the training data...
Number of comments: 5,906



,comment,label
5590,-*-*-*- Will I even be qualified by then?,1
1589,-*-*-*- God doesn't exist and you're living yo...,0
3464,"-*-*-*- I was raised in Argentina, but moved t...",1
1418,Oh dear only in in the east,0
3698,A lot of people are gay and they are frightene...,1


In [15]:
import gc

comments = df.comment.values
labels = df.label.values

# Tokenize all of the comments and map the tokens to their word IDs
input_ids=[]
input_labels=[]

for i,comment in enumerate(comments):
  # (1) Tokenize the comment.
  # (2) Prepend the '[CLS]' token to the start and append the '[SEP]' token to the end
  # (3) Map tokens to their IDs

  if isinstance(comment, float):
    continue

  tokenized_comment = tokenizer.tokenize(comment)
  tokenized_comment.insert(0, '[CLS]')
  tokenized_comment.append('[SEP]')

  if len(tokenized_comment) > 512:
    continue
  
  comment_ids = tokenizer.convert_tokens_to_ids(tokenized_comment)
  input_ids.append(comment_ids)
  input_labels.append(labels[i])
  
# Print the comment 0, now as a list of IDs
print('Number of comments in dataset: ', len(input_ids))
hate_count = 0
for label in input_labels:
  if label == 0:
    hate_count += 1
print('Percentage of hate comments in dataset: {:.3f}'.format(hate_count / len(input_ids)))
print('Max comment length: ', max([len(sen) for sen in input_ids]))

# Delete the comments and labels so I free memory so I can train normally
del comments
del labels
gc.collect()

Number of comments in dataset:  5895
Percentage of hate comments in dataset: 0.253
Max comment length:  446


0